### Calculo de Ratios

#### liquidez
* Razón corriente = Activos corrientes / Pasivos corrientes
* Prueba ácida = (Activo corriente - Inventarios) / Pasivo corriente

#### Solvencia y endeudamiento
* Endeudamiento total = Total de pasivos / Total de activos
* Endeudamiento de largo plazo = Pasivo no corriente / Total de activos
* Cobertura de intereses = EBIT / Gastos por intereses

#### Rentabilidad
* ROA (Return on Assets) = Beneficio neto / Activo total
* ROE (Return on Equity) =  Beneficio neto / Patrimonio neto
* Margen de utilidad = 	Beneficio neto / Ventas (ingresos totales) (escrapeado "Margen del beneficio neto %")
* Rotación de activos = ingresos totales / Total de activos
* Rotación de inventarios = ingresos totales / Inventario

In [7]:
import sqlite3
import pandas as pd
pd.options.display.max_rows = 999

# Consultar datos
conn = sqlite3.connect('webmining.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM datos_anuales")
column_names = [description[0] for description in cursor.description]
filas = cursor.fetchall()

# Guardar los datos en un dataframe
df = pd.DataFrame(filas, columns=column_names)

In [8]:
indicadores = df.groupby(["informe","seccion","indicador"]).size()

print(indicadores)

informe     seccion            indicador                                                 
balance     activo_corriente   Crecimiento de efectivo y equivalentes                        74
                               Créditos totales                                              74
                               Efectivo restringido                                          29
                               Efectivo y equivalentes                                       74
                               Gastos pagados por adelantado                                 42
                               Inventario                                                    66
                               Inversiones a corto plazo                                     63
                               Otros activos corrientes                                      66
                               Otros créditos                                                41
                               Total de cuenta

In [9]:
# rubros que componen cada seccion
# no se incluye por ejemplo el rubro "otros creditos" ya que forma parte de "Creditos totales"
activo_corriente = [
    "Efectivo y equivalentes",
    "Inversiones a corto plazo",
    "Créditos totales",
    "Inventario",
    "Gastos pagados por adelantado",
    "Otros activos corrientes"
]
activo_no_corriente = [
    "Planta, propiedad y equipo, neto",
    "Inversiones a largo plazo",
    "Activos intangibles",
    "Total de otros activos"
]
pasivo_corriente = [
    "Total de cuentas por pagar",
    "Total de gastos devengados",
    "Préstamos a corto plazo",
    "Parte actual de la deuda a largo plazo/arrendamientos",
    "Total de otros pasivos corrientes"
]
pasivo_no_corriente = [
    "Deuda a largo plazo",
    "Arrendamientos a largo plazo",
    "Total de otros pasivos"
]
patrimonio_neto = [
    "Total de acciones preferentes",
    "Acciones ordinarias y APIC",
    "Ganancias retenidas",
    "Acciones propias y otras",
    "Intereses minoritarios"
]

inventario = ["Inventario"]

EBIT = ["EBIT"]
	
intereses = ["Total de gastos de intereses"]

In [10]:
# features de indicadores ya escrapeados

indicadores_scrapeados = [
  "Crecimiento de efectivo y equivalentes",
  "Crecimiento de activos totales",
  "Crecimiento del pasivo total",
  "Crecimiento de la deuda total",
  "Crecimiento del beneficio neto",
  "Crecimiento del EBITDA",
  "Crecimiento de los ingresos totales"
]

In [19]:
ac = pd.pivot_table(df[df.indicador.isin(activo_corriente)], index=["company","ejercicio"], values="valor",aggfunc="sum").rename(columns={"valor":"activo_corriente"})
anc = pd.pivot_table(df[df.indicador.isin(activo_no_corriente)], index=["company","ejercicio"], values="valor",aggfunc="sum").rename(columns={"valor":"activo_no_corriente"})
pc = pd.pivot_table(df[df.indicador.isin(pasivo_corriente)], index=["company","ejercicio"], values="valor",aggfunc="sum").rename(columns={"valor":"pasivo_corriente"})
pnc = pd.pivot_table(df[df.indicador.isin(pasivo_no_corriente)], index=["company","ejercicio"], values="valor",aggfunc="sum").rename(columns={"valor":"pasivo_no_corriente"})
pn = pd.pivot_table(df[df.indicador.isin(patrimonio_neto)], index=["company","ejercicio"], values="valor",aggfunc="sum").rename(columns={"valor":"patrimonio_neto"})
inv = pd.pivot_table(df[df.indicador.isin(inventario)], index=["company","ejercicio"], values="valor",aggfunc="sum").rename(columns={"valor":"inventario"})
ebit = pd.pivot_table(df[df.indicador.isin(EBIT)], index=["company","ejercicio"], values="valor",aggfunc="sum").rename(columns={"valor":"EBIT"})
int = pd.pivot_table(df[df.indicador.isin(intereses)], index=["company","ejercicio"], values="valor",aggfunc="sum").rename(columns={"valor":"intereses"})

data = pd.concat([ac,anc,pc,pnc,pn,inv,ebit,int], axis=1)
data.round(2)

activo_corriente  activo_no_corriente  \
company                ejercicio                                          
3m-co                  2020                14975.0              32362.0   
                       2021                15403.0              31669.0   
                       2022                14688.0              31767.0   
                       2023                16379.0              34201.0   
                       2024                15884.0              23984.0   
amazon-com-inc         2020               132733.0             188462.0   
                       2021               161580.0             258969.0   
                       2022               146791.0             315884.0   
                       2023               172351.0             355503.0   
                       2024               190867.0             434027.0   
apple-computer-inc     2020               143713.0             180175.0   
                       2021               134836.0             216166.0   
                       2022               135405.0             217350.0   
                       2023               143566.0             209017.0   
                       2024               152987.0             211993.0   
berkshire-hathaway-inc 2020               194026.0             679703.0   
                       2021               208089.0             750695.0   
                       2022               203413.0             745052.0   
                       2023               245712.0             824266.0   
                       2024               407447.0             746434.0   
chevron                2020                26078.0             213712.0   
                       2021                33738.0             205797.0   
                       2022                50343.0             207366.0   
                       2023                41128.0             220504.0   
                       2024                40911.0             216027.0   
coca-cola-co           2020                19240.0              68056.0   
                       2021                22544.0              71809.0   
                       2022                22591.0              70172.0   
                       2023                26732.0              70971.0   
                       2024                25997.0              74552.0   
disney                 2020                35251.0             166298.0   
                       2021                33657.0             169952.0   
                       2022                29098.0             174533.0   
                       2023                32763.0             172816.0   
                       2024                25241.0             170978.0   
google-inc             2020               174296.0             145320.0   
                       2021               188143.0             171125.0   
                       2022               164795.0             200469.0   
                       2023               171530.0             230862.0   
                       2024               163711.0             286545.0   
johnson-johnson        2021                51237.0             123657.0   
                       2022                60979.0             121039.0   
                       2023                49464.0             132084.0   
                       2024                55893.0             124211.0   
microsoft-corp         2020               181915.0             119396.0   
                       2021               184406.0             149373.0   
                       2022               169684.0             195156.0   
                       2023               184257.0             227719.0   
                       2024               159734.0             352429.0   
nike                   2020                20556.0              10786.0   
                       2021                26291.0              11449.0   
                       2022                28213.0         

In [17]:
df[
  (df.indicador.isin(activo_corriente)) &
  (df.company == "apple-computer-inc") &
  (df.ejercicio == 2020)
   ]

,id,company,informe,ejercicio,seccion,indicador,valor
173,8872,apple-computer-inc,balance,2020,activo_corriente,Efectivo y equivalentes,38016.0
175,8874,apple-computer-inc,balance,2020,activo_corriente,Inversiones a corto plazo,52927.0
176,8875,apple-computer-inc,balance,2020,activo_corriente,Créditos totales,37445.0
179,8878,apple-computer-inc,balance,2020,activo_corriente,Inventario,4061.0
180,8879,apple-computer-inc,balance,2020,activo_corriente,Otros activos corrientes,11264.0


In [9]:
df[
(df["informe"] == "balance") &
(df["seccion"] == "patrimonio_neto") &
(df["ejercicio"] == 2020) &
(df["company"] == "apple-computer-inc")
].to_excel("apple_2020.xlsx", index=False)


In [3]:
df["company"].unique()

array(['apple-computer-inc', 'microsoft-corp', 'google-inc',
       'tesla-motors', 'visa-inc', 'berkshire-hathaway-inc',
       'johnson-johnson', 'pfizer', 'amazon-com-inc', 'disney', 'nike',
       'procter-gamble', 'chevron', '3m-co', 'coca-cola-co'], dtype=object)

In [7]:
df["seccion"].unique()

array(['activo_corriente', 'activo_total', 'pasivo_corriente',
       'pasivo_total', 'patrimonio_neto', 'crecimiento_deuda', ''],
      dtype=object)